In [ ]:
%env MAKEFLAGS="-j$(nproc)"
!pip install autoawq --break-system-packages
!pip install --upgrade transformers --break-system-packages

In [ ]:
!pip install autoawq transformers accelerate xformers flash-attn --upgrade --break-system-packages

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated'
quant_path = 'quantized_awq_model'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

In [6]:
%%capture
# setting write token
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfeGpka1BvTHR0UHRnVnBXSERqbkFNR3B6b3VNb3ZVYW5saw==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_HF_TRANSFER=1

In [ ]:
!huggingface-cli repo create Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit --type model -y

In [ ]:
!git lfs install
!git clone https://huggingface.co/ivang71/Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit
%cd Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit
!cp -r ../quantized_awq_model/* .
!git add .
!git commit -m "add model"
!git push
%cd ..

In [ ]:
%cd Qwen2.5-7B-Instruct-Abliterated-AWQ-4bit
!git push
%cd ..

In [ ]:
!pip install bitsandbytes git+https://github.com/huggingface/accelerate.git --break-system-packages

In [ ]:
!git lfs clone https://huggingface.co/huihui-ai/gemma-3-12b-it-abliterated